In [1]:
### Definition of some global parameters.
K = 5  # Number of centroids
RUNS = 25  # Number of K-means runs that are executed in parallel. Equivalently, number of sets of initial points
RANDOM_SEED = 60295531
converge_dist = 0.1 # The K-means algorithm is terminated when the change in the location 
                    # of the centroids is smaller than 0.1

In [2]:
import numpy as np
import pickle
import sys
from numpy.linalg import norm



def print_log(s):
    sys.stdout.write(s + "\n")
    sys.stdout.flush()


def parse_data(row):
    '''
    Parse each pandas row into a tuple of (station_name, feature_vec),
    where feature_vec is the concatenation of the projection vectors
    of TAVG, TRANGE, and SNWD.
    '''
    return (row[0], np.concatenate([row[1], row[2], row[3]]))
    # change the input data into the form (station_name, feature_vec)


def compute_entropy(d):
    '''
    Compute the entropy given the frequency vector `d`
    '''
    d = np.array(d)
    d = 1.0 * d / d.sum()
    return -np.sum(d * np.log2(d))


def choice(p):
    '''
    Generates a random sample from [0, len(p)),
    where p[i] is the probability associated with i. 
    So here p is a vector, p[0],p[1],...,p[n-1]
    '''
    random = np.random.random()  # gives a random value between (0,1)
    r = 0.0
    for idx in range(len(p)):
        r = r + p[idx]
        if r > random:
            return idx 
            # here return the index of the data where indices smaller than this with prob 
            # larger than random
    assert(False)


def kmeans_init(rdd, K, RUNS, seed):
    '''
    Select `RUNS` sets of initial points for `K`-means++
    '''
    # the `centers` variable is what we want to return
    
    n_data = rdd.count()
    shape = rdd.take(1)[0][1].shape[0]
    
    # rdd.take(1) is a tuple (station_name, feature_vec), and we need the dimension of the 
    # feature_vec since it is also the dimension of the centers. 
    
    centers = np.zeros((RUNS, K, shape)) 
    # RUNS record the run round and centers record every update from Round-1 to Round-RUNS.
    
    def update_dist(vec, dist, k):
        new_dist = norm(vec - centers[:, k], axis=1)**2
        return np.min([dist, new_dist], axis=0)


    # The second element `dist` in the tuple below is the closest distance from
    # each data point to the selected points in the initial set, where `dist[i]`
    # is the closest distance to the points in the i-th initial set. where i belongs to 
    # (0,1,2,...,RUNS-1)
    
    data = rdd.map(lambda p: (p, [np.inf] * RUNS)).cache()
    # Let the initial distance at all rounds to be inf, and update each round

    # Collect the feature vectors of all data points beforehand, might be
    # useful in the following for-loop
    
    local_data = rdd.map(lambda (name, vec): vec).collect()

    # Randomly select the first point for every run of k-means++,
    # i.e. randomly select `RUNS` points and add it to the `centers` variable
    
    sample = [local_data[k] for k in np.random.randint(0, len(local_data), RUNS)]
    centers[:, 0] = sample
    
    D = (np.inf)*np.ones((RUNS,n_data)) # my code
    
    for idx in range(K - 1):
        ##############################################################################
        # Insert your code here:
        ##############################################################################
        # In each iteration, you need to select one point for each set
        # of initial points (so select `RUNS` points in total).
        # For each data point x, let D_i(x) be the distance between x and
        # the nearest center that has already been added to the i-th set.
        # Choose a new data point for i-th set using a weighted probability
        # where point x is chosen with probability proportional to D_i(x)^2
        ##############################################################################
        #--------------------   My code starts -------------------------
        D = np.array([update_dist(vec,D[:,x],idx) for x, vec in enumerate(local_data)]).T  
        prob = ((D.T)/D.sum(axis=1)).T                                                  
        for i in range(RUNS):                                         
            ind = choice(prob[i])
            centers[i,idx+1,:] = local_data[ind]
        #--------------------   My code ends ---------------------------
    return centers


def get_closest(p, centers):
    '''
    Return the indices the nearest centroids of `p`.
    `centers` contains sets of centroids, where `centers[i]` is
    the i-th set of centroids.
    '''
    best = [0] * len(centers)    # len(centers) = RUNS
    closest = [np.inf] * len(centers)
    for idx in range(len(centers)):    # idx = 0,1,2,...,RUNS-1
        for j in range(len(centers[0])):   # j = 0,1,2,...,K-1
            temp_dist = norm(p - centers[idx][j])
            if temp_dist < closest[idx]:
                closest[idx] = temp_dist
                best[idx] = j
    return best  # best is a 1*RUNS array, contains the group index of a given vector p


def kmeans(rdd, K, RUNS, converge_dist, seed):
    '''
    Run K-means++ algorithm on `rdd`, where `RUNS` is the number of
    initial sets to use.
    '''
    k_points = kmeans_init(rdd, K, RUNS, seed)
    print_log("Initialized.")
    temp_dist = 1.0

    iters = 0
    st = time.time()
    while temp_dist > converge_dist:
        ##############################################################################
        # INSERT YOUR CODE HERE
        local_data = rdd.map(lambda (name, vec): vec).collect()
        ##############################################################################
        
        # Update all `RUNS` sets of centroids using standard k-means algorithm
        # Outline:
        #   - For each point x, select its nearest centroid in i-th centroids set
        #   - Average all points that are assigned to the same centroid
        #   - Update the centroid with the average of all points that are assigned to it
        
        # Insert your code here
        B = np.array([get_closest(vec,k_points) for vec in local_data]).T  
        
        
        #  
        # new_points = new centers
        


        # You can modify this statement as long as `temp_dist` equals to
        # max( sum( l2_norm of the movement of j-th centroid in each centroids set ))
        
        ##############################################################################

        temp_dist = np.max([
                np.sum([norm(k_points[idx][j] - new_points[(idx, j)]) for j in range(K)])
                    for idx in range(RUNS)])

        iters = iters + 1
        if iters % 5 == 0:
            print_log("Iteration %d max shift: %.2f (time: %.2f)" %
                      (iters, temp_dist, time.time() - st))
            st = time.time()

        # update old centroids
        # You modify this for-loop to meet your need
        for ((idx, j), p) in new_points.items():
            # Finally let k_points = new_points
            k_points[idx][j] = p

    return k_points

In [5]:
data = pickle.load(open("./stations_projections.pickle", "rb"))
rdd = sc.parallelize([parse_data(row[1]) for row in data.iterrows()])
rdd.take(1)

[(u'USC00044534', array([  3.04796236e+03,   1.97434852e+03,   1.50560792e+02,
          -2.90363288e+03,  -2.36907268e+02,   1.47021791e+02,
           1.91503001e-01,   1.87262808e-01,  -4.01379553e-02]))]

In [6]:
seed = np.random.seed(RANDOM_SEED)

In [9]:
# This cell test if the initialization function works
import time
st = time.time()
center = kmeans_init(rdd, K, RUNS, seed)
print np.shape(center)
print "Time takes to converge:", time.time() - st

(25, 5, 9)
Time takes to converge: 1.78669595718


In [7]:
# This cell test if the initialization function works
print center

[[[  1.49010782e+03   1.94542355e+03   1.53683044e+02 ...,   2.20539692e+02
     1.22748994e+02   4.13202539e+01]
  [  6.36495448e+02   1.13508697e+03   1.58627087e+00 ...,   3.76151648e+04
    -1.35560407e+04   4.05190927e+03]
  [  2.45777109e+03   1.91940921e+03   8.62422512e+01 ...,   1.87438742e+01
     8.82612480e-01   1.69554224e+00]
  [  4.51056147e+02   2.00082895e+03  -6.13059147e+01 ...,   1.05829752e+04
    -9.02142671e+02   6.00534013e+02]
  [ -1.19636689e+01   1.64470736e+03  -1.79876514e+01 ...,   5.89415987e+03
    -1.12248908e+03   3.35610514e+02]]

 [[  2.23643114e+03   1.95170147e+03  -3.61944620e+01 ...,   6.34745555e+01
     2.10312468e+01  -2.76628330e+00]
  [  1.00620820e+03   1.51910127e+03   1.02186191e+02 ...,   7.31306416e+03
    -7.29344006e+01   2.39440682e+01]
  [  2.12907689e+03   1.39902211e+03   1.12658115e+02 ...,   6.49062252e+00
     2.78814746e+00   2.94524554e+00]
  [  3.79211741e+02   2.41464461e+03   9.62494430e+01 ...,   1.77513598e+03
     2.456

In [15]:
centers = np.array(center)
print np.shape(centers)
print len(centers)
print len(centers[0])

(25, 5, 9)
25
5
